In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [3]:
class DeepQNetwork(nn.Module):
    def __init__(self, ALPHA):
        super(DeepQNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 31, kernel_size=8, stride=4, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.fc1 = nn.Linear(128 * 19 * 8, 512)
        self.fc2 = nn.Linear(512, 6)
        
        self.optimizer = optim.RMSprop(self.parameters(), lr = ALPHA)
        self.loss = nn.MSELoss()
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        
    def forward(self, observation):
        observation = torch.Tensor(observation).to(self.device)
        observation = observation.view(-1, 1, 185, 95)
        observation = torch.relu(self.conv1(observation))
        observation = torch.relu(self.conv2(observation))
        observation = torch.relu(self.conv3(observation))
        
        observation = observation.view(-1, 128*19*8)
        observation = torch.relu(self.fc1(observation))
        action = self.fc2(observation)
        
        return action   
        
        

In [ ]:
class Agent(object):
    def __init__(self, ALPHA, GAMMA, EPSILON, EPSILON_DECAY,
                 MAX_MEMORY_SIZE, EPSILON_END = 0.5,REPLACE = 1000,
                 ACTION_SPACE = [0,1,2,3,4,5]):
        self.ALPHA = ALPHA
        self.GAMMA = GAMMA
        self.EPSILON = EPSILON
        self.EPSILON_DECAY = EPSILON_DECAY
        
        